In [1]:
import numpy as np
import cv2

point = ()
point1 = ()
point2 = ()
point3 = ()
point4 = ()

click_count = 0
i = 1
ids=1 

def on_mouse(event, x, y, flags, params):
    global i,point1,point2,point3,point4,point
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x,y)
        if i == 1:
            point1 = (x,y)
        elif i == 2:
            point2 = (x,y)
        elif i == 3:
            point3 = (x,y)
        elif i == 4:
            point4 = (x,y)
            i = 0
        i = i + 1


# load the image, clone it, and setup the mouse callback function
img = cv2.imread('Challenge4_parking.jpg')
cv2.namedWindow("image")
cv2.setMouseCallback("image", on_mouse)

while True:
    if len(point) != 0 :
        cv2.circle(img,point,3,(255,0,0),-1)
    if len(point4) != 0:
        p = np.array((point1,point2,point3,point4), np.int32)
        
        
        output = open("coordinates.txt", "a")
        
        
        output.write( "-\n          id: " + str(ids) + "\n          points: [" +
                          "[" + str(p[0][0]) + "," + str(p[0][1]) + "]," +
                          "[" + str(p[1][0]) + "," + str(p[1][1]) + "]," +
                          "[" + str(p[2][0]) + "," + str(p[2][1]) + "]," +
                          "[" + str(p[3][0]) + "," + str(p[3][1]) + "]]\n")
        output.close()
        
        p = p.reshape((-1,1,2))
        cv2.drawContours(img,
                         [p],
                         contourIdx=-1,
                         color=(255,0,0),
                         thickness=2,
                         lineType=cv2.LINE_8)
        point1 = ()
        point2 = ()
        point3 = ()
        point4 = ()
        ids+=1
        

    cv2.imshow("image", img)
    
    key = cv2.waitKey(1) & 0xFF 
    if key == ord("q"):
        cv2.destroyAllWindows()
        break
        
 
cv2.destroyAllWindows()

In [2]:
import yaml
import cv2
import numpy as np
img=cv2.imread('Challenge4_parking.jpg')
fn_yaml = r"coordinates.txt"

with open(fn_yaml, 'r') as stream:
    parking_data = yaml.full_load(stream)
    
def check(imgPro):
    spaceCount = 0
    contourIdx= -1
    parking_bounding_rects = []
    for ind,park in enumerate(parking_data):
        points = np.array(park['points'])
        rect = cv2.minAreaRect(points)
        rect = cv2.boxPoints(rect)
        rect = np.int0(rect)
        parking_bounding_rects.append(rect)
        rect = parking_bounding_rects[ind]
        roi_gray = imgPro[rect[1][1]:rect[0][1],
                          rect[1][0]:rect[2][0]]
        count = cv2.countNonZero(roi_gray)
        if count<600:
            spaceCount+=1
            color=(0,0,255)
            thickness=3
        else:
            color=(255,0,0)
            thickness=3
            
        cv2.drawContours(img,[rect],contourIdx,color,thickness)
    cv2.rectangle(img,(45,30),(300,75),(180,0,180),-1)
    cv2.putText(img,f'Parking Space: {spaceCount}',(50,60),cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),2)
        
while True:

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(3,3),1)
    (ret1, Thre)=cv2.threshold(blur, 50,255, cv2.THRESH_BINARY_INV)
    kernel=np.ones((3,3),np.uint8)
    dilate=cv2.dilate(Thre,kernel,iterations=1)
    check(dilate)

    cv2.imshow("Image",img)
    key = cv2.waitKey(1) & 0xFF 
    if key == ord("q"):
        cv2.destroyAllWindows()
        break
            
